# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770500852890                   -0.52    9.0         
  2   -2.771693099565       -2.92       -1.32    1.0    197ms
  3   -2.771714389768       -4.67       -2.48    1.0    134ms
  4   -2.771714639197       -6.60       -3.19    1.0    185ms
  5   -2.771714712678       -7.13       -3.82    2.0    214ms
  6   -2.771714715074       -8.62       -4.58    1.0    142ms
  7   -2.771714715248       -9.76       -5.30    2.0    166ms
  8   -2.771714715249      -12.02       -5.85    1.0    168ms
  9   -2.771714715250      -12.25       -7.17    2.0    176ms
 10   -2.771714715250      -13.85       -6.98    2.0    183ms
 11   -2.771714715250   +  -13.89       -8.34    1.0    161ms


-0.0001345739782071242

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770243474096                   -0.53    9.0         
  2   -2.771769645752       -2.82       -1.29    1.0    141ms
  3   -2.771801042350       -4.50       -2.73    1.0    193ms
  4   -2.771802070104       -5.99       -3.75    2.0    208ms
  5   -2.771802074239       -8.38       -4.27    1.0    144ms
  6   -2.771802074459       -9.66       -5.54    1.0    182ms
  7   -2.771802074474      -10.83       -5.34    2.0    164ms
  8   -2.771802074476      -11.74       -6.00    1.0    151ms
  9   -2.771802074476      -13.08       -6.54    1.0    180ms
 10   -2.771802074476      -13.62       -7.12    2.0    189ms
 11   -2.771802074476      -14.35       -7.87    1.0    163ms
 12   -2.771802074476      -13.95       -8.19    1.0    165ms


0.017612221621013592

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345739782071242
Displaced dipole:  0.017612221621013592
Polarizability :   1.7746795599220717


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920664580e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551072173e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852762152151e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593965945e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787721699e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.275623579209e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.413622141007e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.413622141007e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.790727646286e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.819972179204e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 5.078883245359e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.220008894920e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.220008894920e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.